In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.model_selection import KFold
import numpy as np
import cv2 as cv
import os
import keras

In [ ]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170508288/170498071 [==============================] - 11s 0us/step


In [ ]:
print(X_train.shape)

(50000, 32, 32, 3)


In [ ]:
DataDir = "/content/drive/MyDrive/FEC - version 2/CK+48" # training dataset
Classes = ["anger", "contempt", "disgust", "fear", "happy", "sadness", "surprise"] # list of classes
from tensorflow.keras.utils import to_categorical

def create_training_data():
  X, y = [], []
  for Class in Classes:
      path = os.path.join(DataDir, Class)
      class_num = Classes.index(Class) # Label
      for img in os.listdir(path):
          try:
              img_array = cv.imread(os.path.join(path, img), 0)
              X.append(img_array)
              y.append(class_num)
          except Exception as e:
              pass

  return np.array(X), to_categorical(np.array(y), num_classes=7)

X, y = create_training_data()

In [ ]:
X.shape

(981, 48, 48)

In [ ]:
y.shape

(981, 7)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator 
datagen = ImageDataGenerator( 
    rescale=1./255,
    rotation_range = 10,
    horizontal_flip = True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    fill_mode = 'nearest')

In [ ]:
X = X.reshape(X.shape[0], 48, 48, 1)
datagen.fit(X)

In [ ]:
print(X.shape)

(981, 48, 48, 1)


In [ ]:
train_flow = datagen.flow(X, y, batch_size=32) 

In [ ]:
print(train_flow)

In [ ]:
from tensorflow.keras.utils import plot_model
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.regularizers import l1, l2
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix

def get_model(input_shape=(48,48,1)):
# first input model
  visible = Input(shape=input_shape, name='input')
  num_classes = 7

  #the 1-st block
  conv1_1 = Conv2D(64, kernel_size=3, activation='relu', padding='same', name = 'conv1_1')(visible)
  conv1_1 = BatchNormalization()(conv1_1)
  conv1_2 = Conv2D(64, kernel_size=3, activation='relu', padding='same', name = 'conv1_2')(conv1_1)
  conv1_2 = BatchNormalization()(conv1_2)
  pool1_1 = MaxPooling2D(pool_size=(2,2), name = 'pool1_1')(conv1_2)
  drop1_1 = Dropout(0.3, name = 'drop1_1')(pool1_1)

  #the 2-nd block
  conv2_1 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name = 'conv2_1')(drop1_1)
  conv2_1 = BatchNormalization()(conv2_1)
  conv2_2 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name = 'conv2_2')(conv2_1)
  conv2_2 = BatchNormalization()(conv2_2)
  conv2_3 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name = 'conv2_3')(conv2_2)
  conv2_2 = BatchNormalization()(conv2_3)
  pool2_1 = MaxPooling2D(pool_size=(2,2), name = 'pool2_1')(conv2_3)
  drop2_1 = Dropout(0.3, name = 'drop2_1')(pool2_1)

  #the 3-rd block
  conv3_1 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_1')(drop2_1)
  conv3_1 = BatchNormalization()(conv3_1)
  conv3_2 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_2')(conv3_1)
  conv3_2 = BatchNormalization()(conv3_2)
  conv3_3 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_3')(conv3_2)
  conv3_3 = BatchNormalization()(conv3_3)
  conv3_4 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_4')(conv3_3)
  conv3_4 = BatchNormalization()(conv3_4)
  pool3_1 = MaxPooling2D(pool_size=(2,2), name = 'pool3_1')(conv3_4)
  drop3_1 = Dropout(0.3, name = 'drop3_1')(pool3_1)

  #the 4-th block
  conv4_1 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_1')(drop3_1)
  conv4_1 = BatchNormalization()(conv4_1)
  conv4_2 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_2')(conv4_1)
  conv4_2 = BatchNormalization()(conv4_2)
  conv4_3 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_3')(conv4_2)
  conv4_3 = BatchNormalization()(conv4_3)
  conv4_4 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_4')(conv4_3)
  conv4_4 = BatchNormalization()(conv4_4)
  pool4_1 = MaxPooling2D(pool_size=(2,2), name = 'pool4_1')(conv4_4)
  drop4_1 = Dropout(0.3, name = 'drop4_1')(pool4_1)
  
  #the 5-th block
  conv5_1 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_1')(drop4_1)
  conv5_1 = BatchNormalization()(conv5_1)
  conv5_2 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_2')(conv5_1)
  conv5_2 = BatchNormalization()(conv5_2)
  conv5_3 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_3')(conv5_2)
  conv5_3 = BatchNormalization()(conv5_3)
  conv5_4 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_4')(conv5_3)
  conv5_3 = BatchNormalization()(conv5_3)
  pool5_1 = MaxPooling2D(pool_size=(2,2), name = 'pool5_1')(conv5_4)
  drop5_1 = Dropout(0.3, name = 'drop5_1')(pool5_1)
  
  #Flatten and output
  flatten = Flatten(name = 'flatten')(drop5_1)
  ouput = Dense(num_classes, activation='softmax', name = 'output')(flatten)
  
  # create model 
  model = Model(inputs =visible, outputs = ouput)
  
  # summary layers
  print(model.summary())
  
  return model

In [ ]:
def get_model_VGG():
  model = Sequential()
  model.add(ZeroPadding2D((1,1),input_shape=(48,48,1)))
  model.add(Convolution2D(64, 3, 3, activation='relu'))
  model.add(ZeroPadding2D((1,1)))
  model.add(Convolution2D(64, 3, 3, activation='relu'))
  model.add(MaxPooling2D((2,2), strides=(2,2)))

  model.add(ZeroPadding2D((1,1)))
  model.add(Convolution2D(128, 3, 3, activation='relu'))
  model.add(ZeroPadding2D((1,1)))
  model.add(Convolution2D(128, 3, 3, activation='relu'))
  model.add(MaxPooling2D((2,2), strides=(2,2)))

  model.add(ZeroPadding2D((1,1)))
  model.add(Convolution2D(256, 3, 3, activation='relu'))
  model.add(ZeroPadding2D((1,1)))
  model.add(Convolution2D(256, 3, 3, activation='relu'))
  model.add(ZeroPadding2D((1,1)))
  model.add(Convolution2D(256, 3, 3, activation='relu'))
  model.add(MaxPooling2D((2,2), strides=(2,2)))

  model.add(ZeroPadding2D((1,1)))
  model.add(Convolution2D(512, 3, 3, activation='relu'))
  model.add(ZeroPadding2D((1,1)))
  model.add(Convolution2D(512, 3, 3, activation='relu'))
  model.add(ZeroPadding2D((1,1)))
  model.add(Convolution2D(512, 3, 3, activation='relu'))
  model.add(MaxPooling2D((2,2), strides=(2,2)))


  model.add(ZeroPadding2D((1,1)))
  model.add(Convolution2D(512, 3, 3, activation='relu'))
  model.add(ZeroPadding2D((1,1)))
  model.add(Convolution2D(512, 3, 3, activation='relu'))
  model.add(ZeroPadding2D((1,1)))
  model.add(Convolution2D(512, 3, 3, activation='relu'))
  model.add(MaxPooling2D((2,2), strides=(2,2)))

  model.add(Flatten())
  model.add(Dense(4096, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(4096, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(7, activation='softmax'))

  model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])
  
  return model

In [ ]:
BATCH_SIZE, num_epochs, num_folds = 32, 30, 10

X = X.astype('float32')
X = X / 255

accuracy_list, loss_list = [], []
# Định nghĩa K-Fold CV
kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_idx = 1

for train_ids, val_ids in kfold.split(X, y):

  model = get_model()

  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  
  print("Bắt đầu train Fold ", fold_idx)

  # Train model
  model.fit(X[train_ids], y[train_ids],
              batch_size=BATCH_SIZE,
              epochs=num_epochs,
            verbose=1)

  # Test và in kết quả
  scores = model.evaluate(X[val_ids], y[val_ids], verbose=0)
  print("Đã train xong Fold ", fold_idx)

  # Thêm thông tin accuracy và loss vào list
  accuracy_list.append(scores[1] * 100)
  loss_list.append(scores[0])

  # Sang Fold tiếp theo
  fold_idx = fold_idx + 1

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 48, 48, 1)]       0         
                                                                 
 conv1_1 (Conv2D)            (None, 48, 48, 64)        640       
                                                                 
 batch_normalization_17 (Bat  (None, 48, 48, 64)       256       
 chNormalization)                                                
                                                                 
 conv1_2 (Conv2D)            (None, 48, 48, 64)        36928     
                                                                 
 batch_normalization_18 (Bat  (None, 48, 48, 64)       256       
 chNormalization)                                                
                                                                 
 pool1_1 (MaxPooling2D)      (None, 24, 24, 64)        0   

In [ ]:
# In kết quả tổng thể
print('* Chi tiết các fold')
for i in range(0, len(accuracy_list)):
  print(f'> Fold {i+1} - Loss: {loss_list[i]} - Accuracy: {accuracy_list[i]}%')

print('* Đánh giá tổng thể các folds:')
print(f'> Accuracy: {np.mean(accuracy_list)} (Độ lệch +- {np.std(accuracy_list)})')
print(f'> Loss: {np.mean(loss_list)}')

* Chi tiết các fold
> Fold 1 - Loss: 0.09125568717718124 - Accuracy: 94.94949579238892%
> Fold 2 - Loss: 0.679624080657959 - Accuracy: 83.67347121238708%
> Fold 3 - Loss: 0.21445485949516296 - Accuracy: 92.85714030265808%
> Fold 4 - Loss: 0.4175910949707031 - Accuracy: 86.73469424247742%
> Fold 5 - Loss: 0.1091839149594307 - Accuracy: 93.87755393981934%
> Fold 6 - Loss: 0.14367982745170593 - Accuracy: 95.91836929321289%
> Fold 7 - Loss: 0.47764551639556885 - Accuracy: 85.71428656578064%
> Fold 8 - Loss: 0.0634140595793724 - Accuracy: 98.97959232330322%
> Fold 9 - Loss: 0.22023199498653412 - Accuracy: 93.87755393981934%
> Fold 10 - Loss: 0.7280715703964233 - Accuracy: 74.48979616165161%
* Đánh giá tổng thể các folds:
> Accuracy: 90.10719537734985 (Độ lệch +- 6.9849683195606245)
> Loss: 0.3145152606070042


In [ ]:
model.save('/content/drive/MyDrive/FEC - version 2/CNN_CKplus_file_30epochs.h5')